NHÓM 3

Thành viên nhóm:

Võ Hưng Thạnh - K214141338 (nhóm trưởng)

Trần Bích Quân - K214140951

Trần Minh Châu - K214140934

Nguyễn Thị Mai Vàng - K214142106

Nguyễn Đình Ngân Sơn - K214141337

In [1]:
pip install pyspark


In [2]:
pip install vnstock3 --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 3.0 MB/s eta 0:00:00


In [3]:
import joblib
import os
import pandas as pd
import numpy as np
from vnstock import stock_historical_data
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext
from pyspark.sql import SparkSession, Row
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objs as go
import plotly.subplots as sp
import ipywidgets as widgets
from IPython.display import display, clear_output

from keras.layers import GRU, Dropout, Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, BatchNormalization
from keras.models import Sequential
from keras.callbacks import EarlyStopping

import datetime


spark = SparkSession.builder \
    .appName("Stock Data Processing") \
    .getOrCreate()

stock_symbols = [
    "VCB", "VNM", "MWG", "VIC", "SSI", "DGC", "CTD", "FPT", "MSN",
    "GVR", "GAS", "POW", "HPG", "REE", "DHG", "GMD", "VHC", "KBC",
    "CMG", "VRE"
]

today = datetime.date.today()
four_years_ago = today - datetime.timedelta(days=1461)

all_data = []

for stock_symbol in stock_symbols:
    data = stock_historical_data(
        stock_symbol,
        start_date=four_years_ago.strftime('%Y-%m-%d'),
        end_date=today.strftime('%Y-%m-%d')
    )

    df_pandas = pd.DataFrame(data)[['time', 'open', 'close', 'high', 'low', 'volume']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})
    df_pandas['ticker'] = stock_symbol

    all_data.append(df_pandas)

df_combined = pd.concat(all_data, ignore_index=True)
df_spark = spark.createDataFrame(df_combined)
df_spark.printSchema()
result_df = df_spark.toPandas()
result_df

spark.stop()

**Vui lòng chuyển đổi sang Vnstock3** thế hệ mới (3.1.0) với câu lệnh: `pip install vnstock3 --upgrade`.
**Từ 1/1/2025, vnstock3 sẽ được cài đặt khi sử dụng cú pháp** `pip install vnstock` **thay cho Vnstock Legacy** hiện tại.
Xem chi tiết [chuyển đổi sang vnstock3](https://vnstocks.com/docs/tai-lieu/migration-chuyen-doi-sang-vnstock3).
Phiên bản **Vnstock Legacy (0.2.9.2.3)** bạn đang sử dụng **sẽ không được nâng cấp thêm.**
Từ 7/10/2024 Vnstock giới thiệu nhóm Facebook Cộng đồng Vnstock, tham gia thảo luận tại đây: https://www.facebook.com/groups/vnstock.official

root
 |-- date: date (nullable = true)
 |-- open: long (nullable = true)
 |-- close: long (nullable = true)
 |-- high: long (nullable = true)
 |-- low: long (nullable = true)
 |-- volume: long (nullable = true)
 |-- ticker: string (nullable = true)



In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
from vnstock3 import Vnstock
import logging

spark = SparkSession.builder.appName("VNStockAnalysis").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
logging.getLogger("vnstock3").setLevel(logging.ERROR)

stock = Vnstock().stock()

df_listing = stock.listing.symbols_by_exchange()
df_listing_enhanced = stock.listing.symbols_by_industries()

stock_symbols = [
    "VCB", "VNM", "MWG", "VIC", "SSI", "DGC", "CTD", "FPT", "MSN",
    "GVR", "GAS", "POW", "HPG", "REE", "DHG", "GMD", "VHC", "KBC",
    "CMG", "VRE"
]

df_listing_filtered = df_listing[df_listing['symbol'].isin(stock_symbols)]
df_listing_enhanced_filtered = df_listing_enhanced[df_listing_enhanced['symbol'].isin(stock_symbols)]

rdd_listing = spark.createDataFrame(df_listing_filtered).rdd.map(
    lambda row: (row['symbol'], (row['exchange'], row['organ_short_name'], row['organ_name']))
)
rdd_listing_enhanced = spark.createDataFrame(df_listing_enhanced_filtered).rdd.map(
    lambda row: (row['symbol'], (row['icb_name2'],))
)

rdd_joined = rdd_listing.join(rdd_listing_enhanced)

rdd_indexed = rdd_joined.map(lambda row: (
    row[0],
    row[1][0][0],
    row[1][0][1],
    row[1][0][2],
    row[1][1][0]
)).zipWithIndex().map(lambda row: (row[1] + 1,) + row[0]).cache()

df_indexed = spark.createDataFrame(rdd_indexed, schema=["index", "symbol", "exchange", "organ_short_name", "organ_name", "icb_name2"])
print("Kết quả sau khi xử lý dữ liệu:")
df_indexed.show(truncate=False)

spark.stop()


    Khi tiếp tục sử dụng Vnstock3, bạn xác nhận rằng bạn đã đọc, hiểu và đồng ý với Chính sách quyền riêng tư và Điều khoản, điều kiện về giấy phép sử dụng Vnstock3.

    Chi tiết:

    - Giấy phép sử dụng phần mềm: https://vnstocks.com/docs/tai-lieu/giay-phep-su-dung
    - Chính sách quyền riêng tư: https://vnstocks.com/docs/tai-lieu/chinh-sach-quyen-rieng-tu
    


2024-11-14 13:33:17 - vnstock3.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
INFO:vnstock3.common.vnstock:Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2024-11-14 13:33:17 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Kết quả sau khi xử lý dữ liệu:
+-----+------+--------+-------------------------+------------------------------------------------------+-----------------------------+
|index|symbol|exchange|organ_short_name         |organ_name                                            |icb_name2                    |
+-----+------+--------+-------------------------+------------------------------------------------------+-----------------------------+
|1    |VCB   |HSX     |Vietcombank              |Ngân hàng Thương mại Cổ phần Ngoại thương Việt Nam    |Ngân hàng                    |
|2    |POW   |HSX     |Điện lực Dầu khí Việt Nam|Tổng Công ty Điện lực Dầu khí Việt Nam - CTCP         |Điện, nước & xăng dầu khí đốt|
|3    |GVR   |HSX     |Tập đoàn CN Cao su VN    |Tập đoàn Công nghiệp Cao su Việt Nam - Công ty Cổ phần|Hóa chất                     |
|4    |DHG   |HSX     |Dược Hậu Giang           |Công ty Cổ phần Dược Hậu Giang                        |Y tế                         |
|5    |DGC   |HSX     |H

In [5]:
result_df

,date,open,close,high,low,volume,ticker
0,2020-11-16,56720,55810,57110,55290,1250340,VCB
1,2020-11-17,55810,56980,56980,55620,548250,VCB
2,2020-11-18,56980,56980,57050,56590,906810,VCB
3,2020-11-19,57050,58680,58680,56850,2584440,VCB
4,2020-11-20,59200,60170,60170,58610,1231960,VCB
...,...,...,...,...,...,...,...
19997,2024-11-08,18150,17950,18150,17750,4487200,VRE
19998,2024-11-11,17800,18000,18000,17600,5983800,VRE
19999,2024-11-12,17850,18000,18200,17850,4570800,VRE
20000,2024-11-13,18000,18100,18200,17950,6231800,VRE


In [6]:
spark = SparkSession.builder.appName("Monte Carlo Simulation").getOrCreate()

df_spark = spark.createDataFrame(df_combined)

price_rdd = df_spark.rdd.map(lambda row: (row['ticker'], row['close']))

def monte_carlo_simulation(prices, num_simulations=1000, days=10):
    simulations = []
    last_price = prices[-1]
    for _ in range(num_simulations):
        simulated_prices = [last_price]
        for _ in range(days):
            change = np.random.normal(0, 1)
            simulated_price = simulated_prices[-1] * (1 + change / 100)
            simulated_prices.append(simulated_price)
        simulations.append(simulated_prices)
    return simulations

simulated_rdd = price_rdd \
    .groupByKey() \
    .mapValues(lambda prices: monte_carlo_simulation(list(prices)))

results = simulated_rdd.collect()

for ticker, simulations in results:
    print(f"\nMã cổ phiếu {ticker} - Kết quả Monte Carlo:")
    for sim_idx, simulation in enumerate(simulations[:3], 1):
        print(f"  Mô phỏng {sim_idx}: {simulation}")

spark.stop()


Mã cổ phiếu VCB - Kết quả Monte Carlo:
  Mô phỏng 1: [92000, 91955.92774114311, 90870.47272896355, 91516.50641032257, 91715.31466063406, 91050.32124676224, 90058.97392572546, 90171.20499571579, 91371.52367129938, 91410.06840650487, 91369.35672439887]
  Mô phỏng 2: [92000, 92140.48697345723, 92332.0544712603, 91418.09901369724, 91360.92525060219, 91853.12958287839, 91799.2803332317, 92534.3713100334, 91916.12477523796, 92556.49008070365, 92153.45771214078]
  Mô phỏng 3: [92000, 92252.8481524118, 91714.74539427218, 90336.51603829853, 89529.26949096762, 88694.49071884168, 88375.90036601842, 88750.29156779185, 88858.6474335216, 88014.6292344723, 87625.85936940534]

Mã cổ phiếu VIC - Kết quả Monte Carlo:
  Mô phỏng 1: [40700, 40923.47371283903, 40822.102528522, 41022.233932224764, 41310.23069546436, 41044.88034758182, 39884.98053421745, 39615.49011677359, 39091.12719358367, 39077.691325702945, 39045.77453706532]
  Mô phỏng 2: [40700, 40312.38881775812, 40335.28434999654, 39432.32031139134,

In [7]:
spark = SparkSession.builder.appName("Monte Carlo Simulation").getOrCreate()
sc = spark.sparkContext
rdd = sc.parallelize(result_df.values.tolist())

prices_rdd = rdd.map(lambda row: (row[6], row[2]))

average_close_rdd = prices_rdd \
    .map(lambda x: (x[0], (x[1], 1))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .mapValues(lambda x: round(x[0] / x[1], 2))

prices_grouped_rdd = prices_rdd.groupByKey().mapValues(list)

def monte_carlo_simulation(prices, num_simulations=1000, days=10):
    last_price = prices[-1]
    simulations = []
    for _ in range(num_simulations):
        simulated_prices = [last_price]
        for _ in range(days):
            change = np.random.normal(0, 1)
            simulated_price = simulated_prices[-1] * (1 + change / 100)
            simulated_prices.append(simulated_price)
        simulations.append(simulated_prices)

    increase_prob = sum(1 for sim in simulations if sim[-1] > last_price) / num_simulations
    return increase_prob

increase_prob_rdd = prices_grouped_rdd.mapValues(lambda prices: monte_carlo_simulation(prices))

combined_rdd = average_close_rdd.join(increase_prob_rdd)

decision_rdd = combined_rdd.map(lambda x: (
    x[0],
    x[1][0],
    x[1][1],
    "Buy" if x[1][1] > 0.7 else "Sell" if x[1][1] < 0.3 else "Hold"
))

final_df = decision_rdd.map(lambda x: Row(
    ticker=x[0],
    avg_close=x[1],
    increase_prob=round(x[2], 4),
    recommendation=x[3]
)).toDF()

final_df = final_df.orderBy("ticker")
print("\nBảng kết quả tổng hợp:")
final_df.show(truncate=False)

spark.stop()


Bảng kết quả tổng hợp:
+------+---------+-------------+--------------+
|ticker|avg_close|increase_prob|recommendation|
+------+---------+-------------+--------------+
|CMG   |34667.17 |0.478        |Hold          |
|CTD   |52949.7  |0.478        |Hold          |
|DGC   |72308.92 |0.49         |Hold          |
|DHG   |94323.74 |0.478        |Hold          |
|FPT   |70284.25 |0.478        |Hold          |
|GAS   |72229.54 |0.526        |Hold          |
|GMD   |44645.42 |0.512        |Hold          |
|GVR   |25558.51 |0.487        |Hold          |
|HPG   |25546.53 |0.479        |Hold          |
|KBC   |29579.43 |0.511        |Hold          |
|MSN   |89285.24 |0.487        |Hold          |
|MWG   |53398.81 |0.494        |Hold          |
|POW   |12817.86 |0.526        |Hold          |
|REE   |50076.73 |0.511        |Hold          |
|SSI   |22139.88 |0.49         |Hold          |
|VCB   |75438.24 |0.476        |Hold          |
|VHC   |55582.79 |0.479        |Hold          |
|VIC   |68328.76

In [8]:
models = {}

stock_symbols = ["VCB", "VNM", "MWG", "VIC", "SSI", "DGC", "CTD", "FPT", "MSN",
    "GVR", "GAS", "POW", "HPG", "REE", "DHG", "GMD", "VHC", "KBC",
    "CMG", "VRE"]

today = datetime.date.today()
four_years_ago = today - datetime.timedelta(days=1461)

for stock_symbol in stock_symbols:
    data = stock_historical_data(
        stock_symbol,
        start_date=four_years_ago.strftime('%Y-%m-%d'),
        end_date=today.strftime('%Y-%m-%d')
    )

    if data.empty:
        continue

    df_pandas = pd.DataFrame(data)[['time', 'close']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})
    df_pandas['date'] = pd.to_datetime(df_pandas['date'])
    df_pandas = df_pandas.sort_values(by='date')

    prices = df_pandas['close'].values.reshape(-1, 1)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)

    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]

    def create_dataset(dataset, look_back=5):
        X, y = [], []
        for i in range(len(dataset) - look_back):
            X.append(dataset[i:(i + look_back), 0])
            y.append(dataset[i + look_back, 0])
        return np.array(X), np.array(y)

    X_train, y_train = create_dataset(train_data)
    X_test, y_test = create_dataset(test_data)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    model = Sequential()
    model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(5, 1)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(50)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=2, validation_data=(X_test, y_test), callbacks=[early_stopping])

    y_pred = model.predict(X_test)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv = scaler.inverse_transform(y_pred)

    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r_squared = r2_score(y_test_inv, y_pred_inv)
    mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100

    models[stock_symbol] = {
        'model': model,
        'y_test': y_test_inv.flatten(),
        'y_pred': y_pred_inv.flatten(),
        'dates': df_pandas['date'].values[-len(y_test):],
        'rmse': rmse,
        'mae': mae,
        'r_squared': r_squared,
        'mape': mape
    }



Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


Epoch 1/100
44/44 - 7s - 168ms/step - loss: 0.0184 - val_loss: 0.0065
Epoch 2/100
44/44 - 2s - 44ms/step - loss: 0.0024 - val_loss: 9.3296e-04
Epoch 3/100
44/44 - 1s - 21ms/step - loss: 0.0023 - val_loss: 0.0013
Epoch 4/100
44/44 - 1s - 26ms/step - loss: 0.0021 - val_loss: 8.9236e-04
Epoch 5/100
44/44 - 1s - 28ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 6/100
44/44 - 1s - 15ms/step - loss: 0.0023 - val_loss: 0.0013
Epoch 7/100
44/44 - 1s - 29ms/step - loss: 0.0021 - val_loss: 0.0031
Epoch 8/100
44/44 - 1s - 29ms/step - loss: 0.0024 - val_loss: 0.0022
Epoch 9/100
44/44 - 1s - 28ms/step - loss: 0.0021 - val_loss: 0.0025
Epoch 10/100
44/44 - 1s - 29ms/step - loss: 0.0020 - val_loss: 0.0012
Epoch 11/100
44/44 - 1s - 29ms/step - loss: 0.0023 - val_loss: 9.0429e-04
Epoch 12/100
44/44 - 2s - 40ms/step - loss: 0.0021 - val_loss: 7.5252e-04
Epoch 13/100
44/44 - 1s - 29ms/step - loss: 0.0018 - val_loss: 0.0011
Epoch 14/100
44/44 - 1s - 24ms/step - loss: 0.0017 - val_loss: 0.0011
Epoch 15/100

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


44/44 - 7s - 162ms/step - loss: 0.0209 - val_loss: 7.0267e-04
Epoch 2/100
44/44 - 2s - 54ms/step - loss: 0.0025 - val_loss: 6.8602e-04
Epoch 3/100
44/44 - 1s - 22ms/step - loss: 0.0023 - val_loss: 6.6062e-04
Epoch 4/100
44/44 - 1s - 15ms/step - loss: 0.0025 - val_loss: 8.5472e-04
Epoch 5/100
44/44 - 1s - 15ms/step - loss: 0.0022 - val_loss: 8.6765e-04
Epoch 6/100
44/44 - 1s - 29ms/step - loss: 0.0021 - val_loss: 9.0015e-04
Epoch 7/100
44/44 - 1s - 15ms/step - loss: 0.0021 - val_loss: 7.4202e-04
Epoch 8/100
44/44 - 1s - 28ms/step - loss: 0.0020 - val_loss: 6.1269e-04
Epoch 9/100
44/44 - 1s - 15ms/step - loss: 0.0021 - val_loss: 6.5313e-04
Epoch 10/100
44/44 - 1s - 28ms/step - loss: 0.0021 - val_loss: 5.9710e-04
Epoch 11/100
44/44 - 1s - 16ms/step - loss: 0.0023 - val_loss: 0.0014
Epoch 12/100
44/44 - 1s - 15ms/step - loss: 0.0021 - val_loss: 6.8774e-04
Epoch 13/100
44/44 - 1s - 15ms/step - loss: 0.0021 - val_loss: 6.5592e-04
Epoch 14/100
44/44 - 1s - 16ms/step - loss: 0.0018 - val_loss:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


...

Epoch 1/100
44/44 - 8s - 180ms/step - loss: 0.0295 - val_loss: 8.4501e-04
Epoch 2/100
44/44 - 2s - 43ms/step - loss: 0.0035 - val_loss: 7.5382e-04
Epoch 3/100
44/44 - 1s - 15ms/step - loss: 0.0030 - val_loss: 7.6012e-04
Epoch 4/100
44/44 - 1s - 15ms/step - loss: 0.0029 - val_loss: 0.0019
Epoch 5/100
44/44 - 1s - 29ms/step - loss: 0.0028 - val_loss: 7.2444e-04
Epoch 6/100
44/44 - 2s - 40ms/step - loss: 0.0025 - val_loss: 7.1216e-04
Epoch 7/100
44/44 - 1s - 30ms/step - loss: 0.0029 - val_loss: 8.9211e-04
Epoch 8/100
44/44 - 1s - 19ms/step - loss: 0.0026 - val_loss: 7.2774e-04
Epoch 9/100
44/44 - 1s - 26ms/step - loss: 0.0027 - val_loss: 0.0021
Epoch 10/100
44/44 - 1s - 29ms/step - loss: 0.0026 - val_loss: 6.7421e-04
Epoch 11/100
44/44 - 1s - 16ms/step - loss: 0.0024 - val_loss: 6.5501e-04
Epoch 12/100
44/44 - 1s - 28ms/step - loss: 0.0023 - val_loss: 9.7674e-04
Epoch 13/100
44/44 - 1s - 29ms/step - loss: 0.0026 - val_loss: 6.4584e-04
Epoch 14/100
44/44 - 1s - 29ms/step - loss: 0.0022 - v

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


44/44 - 9s - 193ms/step - loss: 0.0093 - val_loss: 0.0039
Epoch 2/100
44/44 - 1s - 30ms/step - loss: 6.8076e-04 - val_loss: 0.0016
Epoch 3/100
44/44 - 1s - 29ms/step - loss: 7.8312e-04 - val_loss: 0.0016
Epoch 4/100
44/44 - 1s - 16ms/step - loss: 6.6190e-04 - val_loss: 0.0014
Epoch 5/100
44/44 - 1s - 29ms/step - loss: 7.0360e-04 - val_loss: 0.0014
Epoch 6/100
44/44 - 1s - 28ms/step - loss: 6.8504e-04 - val_loss: 0.0018
Epoch 7/100
44/44 - 1s - 28ms/step - loss: 6.9523e-04 - val_loss: 0.0027
Epoch 8/100
44/44 - 1s - 29ms/step - loss: 6.5146e-04 - val_loss: 0.0019
Epoch 9/100
44/44 - 1s - 15ms/step - loss: 5.8785e-04 - val_loss: 0.0024
Epoch 10/100
44/44 - 2s - 40ms/step - loss: 7.1355e-04 - val_loss: 0.0016
Epoch 11/100
44/44 - 1s - 27ms/step - loss: 5.7791e-04 - val_loss: 0.0039
Epoch 12/100
44/44 - 1s - 25ms/step - loss: 6.1581e-04 - val_loss: 0.0026
Epoch 13/100
44/44 - 1s - 19ms/step - loss: 5.7384e-04 - val_loss: 0.0036
Epoch 14/100
44/44 - 1s - 16ms/step - loss: 5.6388e-04 - val_l

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


44/44 - 8s - 189ms/step - loss: 0.0423 - val_loss: 0.0080
Epoch 2/100
44/44 - 1s - 23ms/step - loss: 0.0048 - val_loss: 0.0050
Epoch 3/100
44/44 - 1s - 16ms/step - loss: 0.0043 - val_loss: 0.0041
Epoch 4/100
44/44 - 1s - 29ms/step - loss: 0.0040 - val_loss: 0.0024
Epoch 5/100
44/44 - 1s - 16ms/step - loss: 0.0040 - val_loss: 0.0032
Epoch 6/100
44/44 - 1s - 28ms/step - loss: 0.0040 - val_loss: 0.0016
Epoch 7/100
44/44 - 1s - 29ms/step - loss: 0.0040 - val_loss: 0.0032
Epoch 8/100
44/44 - 1s - 28ms/step - loss: 0.0037 - val_loss: 0.0022
Epoch 9/100
44/44 - 1s - 15ms/step - loss: 0.0042 - val_loss: 0.0018
Epoch 10/100
44/44 - 1s - 29ms/step - loss: 0.0034 - val_loss: 0.0013
Epoch 11/100
44/44 - 1s - 15ms/step - loss: 0.0034 - val_loss: 0.0020
Epoch 12/100
44/44 - 1s - 17ms/step - loss: 0.0034 - val_loss: 0.0021
Epoch 13/100
44/44 - 1s - 26ms/step - loss: 0.0036 - val_loss: 0.0011
Epoch 14/100
44/44 - 1s - 30ms/step - loss: 0.0033 - val_loss: 0.0011
Epoch 15/100
44/44 - 1s - 28ms/step - lo

In [9]:
def plot_technical_indicators(stock_symbol):
    today = datetime.date.today()
    one_year_ago = today - datetime.timedelta(days=1461)

    data = stock_historical_data(
        stock_symbol,
        start_date=one_year_ago.strftime('%Y-%m-%d'),
        end_date=today.strftime('%Y-%m-%d')
    )

    df = pd.DataFrame(data)[['time', 'open', 'close', 'high', 'low', 'volume']]
    df = df.rename(columns={'time': 'date'})
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    df = df.dropna()

    df['date_str'] = df['date'].dt.strftime('%Y-%m-%d')

    df['MA50'] = df['close'].rolling(window=50).mean()
    df['MA100'] = df['close'].rolling(window=100).mean()
    df['BB_upper'] = df['close'].rolling(window=20).mean() + 2 * df['close'].rolling(window=20).std()
    df['BB_lower'] = df['close'].rolling(window=20).mean() - 2 * df['close'].rolling(window=20).std()
    df['MACD'] = df['close'].ewm(span=12).mean() - df['close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()

    delta = df['close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    fig = sp.make_subplots(
        rows=3, cols=1, shared_xaxes=True,
        row_heights=[0.6, 0.2, 0.2], vertical_spacing=0.05,
        specs=[[{"secondary_y": True}], [{}], [{}]]
    )

    fig.add_trace(
        go.Candlestick(
            x=df['date_str'], open=df['open'], high=df['high'],
            low=df['low'], close=df['close'], name='Candlestick'
        ), row=1, col=1, secondary_y=False
    )

    fig.add_trace(
        go.Bar(
            x=df['date_str'], y=df['volume'],
            marker_color='lightblue', opacity=0.5, name='Volume'
        ), row=1, col=1, secondary_y=True
    )


    fig.add_trace(go.Scatter(x=df['date_str'], y=df['MA50'], mode='lines', name='MA50'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['MA100'], mode='lines', name='MA100'), row=1, col=1)

    fig.add_trace(go.Scatter(x=df['date_str'], y=df['BB_upper'], line=dict(color='lightgray'), name='BB Upper'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['BB_lower'], line=dict(color='lightgray'), name='BB Lower'), row=1, col=1)

    fig.add_trace(go.Scatter(x=df['date_str'], y=df['MACD'], mode='lines', name='MACD'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['MACD_Signal'], mode='lines', name='MACD Signal'), row=2, col=1)

    fig.add_trace(go.Scatter(x=df['date_str'], y=df['RSI'], mode='lines', name='RSI'), row=3, col=1)

    fig.add_shape(type='line', x0=df['date_str'].min(), x1=df['date_str'].max(), y0=70, y1=70,
                  line=dict(color='red', dash='dash'), row=3, col=1)
    fig.add_shape(type='line', x0=df['date_str'].min(), x1=df['date_str'].max(), y0=30, y1=30,
                  line=dict(color='green', dash='dash'), row=3, col=1)

    fig.update_layout(
        title=f'{stock_symbol} - Technical Indicators and Volume',
        template='plotly_white',
        height=1000, width=1200,
        hovermode='x unified',
        xaxis=dict(
            type='category',
            tickangle=45
        ),
        yaxis=dict(
            title='Price',
            side='left'
        ),
        yaxis2=dict(
            title='Volume',
            overlaying='y',
            side='right'
        ),
        xaxis_rangeslider_visible=False
    )

    fig.show()



In [10]:
gru_models = {}

for stock_symbol in stock_symbols:
    data = stock_historical_data(stock_symbol, start_date=four_years_ago.strftime('%Y-%m-%d'), end_date=today.strftime('%Y-%m-%d'))

    if data.empty:
        continue

    df_pandas = pd.DataFrame(data)[['time', 'close']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})
    df_pandas['date'] = pd.to_datetime(df_pandas['date'])
    df_pandas = df_pandas.sort_values(by='date')

    prices = df_pandas['close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)

    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]

    X_train, y_train = create_dataset(train_data)
    X_test, y_test = create_dataset(test_data)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    gru_model = Sequential()
    gru_model.add(GRU(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    gru_model.add(Dropout(0.3))
    gru_model.add(GRU(64))
    gru_model.add(Dropout(0.3))
    gru_model.add(Dense(1))
    gru_model.compile(loss='mean_squared_error', optimizer='adam')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    gru_model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])

    y_pred_gru = gru_model.predict(X_test)
    y_test_inv_gru = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv_gru = scaler.inverse_transform(y_pred_gru)

    rmse_gru = np.sqrt(mean_squared_error(y_test_inv_gru, y_pred_inv_gru))
    mae_gru = mean_absolute_error(y_test_inv_gru, y_pred_inv_gru)
    r_squared_gru = r2_score(y_test_inv_gru, y_pred_inv_gru)
    mape_gru = np.mean(np.abs((y_test_inv_gru - y_pred_inv_gru) / y_test_inv_gru)) * 100

    gru_models[stock_symbol] = {
        'model': gru_model,
        'y_test': y_test_inv_gru.flatten(),
        'y_pred': y_pred_inv_gru.flatten(),
        'dates': df_pandas['date'].values[-len(y_test):],
        'rmse': rmse_gru,
        'mae': mae_gru,
        'r_squared': r_squared_gru,
        'mape': mape_gru
    }


Epoch 1/200


Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


22/22 - 4s - 198ms/step - loss: 0.0278 - val_loss: 0.0258 - learning_rate: 0.0010
Epoch 2/200
22/22 - 1s - 47ms/step - loss: 0.0052 - val_loss: 0.0023 - learning_rate: 0.0010
Epoch 3/200
22/22 - 1s - 26ms/step - loss: 0.0039 - val_loss: 8.2030e-04 - learning_rate: 0.0010
Epoch 4/200
22/22 - 1s - 29ms/step - loss: 0.0031 - val_loss: 8.2333e-04 - learning_rate: 0.0010
Epoch 5/200
22/22 - 1s - 29ms/step - loss: 0.0031 - val_loss: 8.1063e-04 - learning_rate: 0.0010
Epoch 6/200
22/22 - 1s - 28ms/step - loss: 0.0030 - val_loss: 7.8642e-04 - learning_rate: 0.0010
Epoch 7/200
22/22 - 0s - 14ms/step - loss: 0.0031 - val_loss: 0.0024 - learning_rate: 0.0010
Epoch 8/200
22/22 - 0s - 15ms/step - loss: 0.0030 - val_loss: 0.0011 - learning_rate: 0.0010
Epoch 9/200
22/22 - 0s - 16ms/step - loss: 0.0029 - val_loss: 9.5563e-04 - learning_rate: 2.0000e-04
Epoch 10/200
22/22 - 0s - 15ms/step - loss: 0.0028 - val_loss: 0.0011 - learning_rate: 2.0000e-04
Epoch 11/200
22/22 - 0s - 15ms/step - loss: 0.0031 -

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


22/22 - 6s - 253ms/step - loss: 0.0364 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 2/200
22/22 - 0s - 22ms/step - loss: 0.0069 - val_loss: 8.8068e-04 - learning_rate: 0.0010
Epoch 3/200
22/22 - 0s - 15ms/step - loss: 0.0036 - val_loss: 6.2915e-04 - learning_rate: 0.0010
Epoch 4/200
22/22 - 0s - 14ms/step - loss: 0.0037 - val_loss: 6.2138e-04 - learning_rate: 0.0010
Epoch 5/200
22/22 - 0s - 16ms/step - loss: 0.0037 - val_loss: 6.1961e-04 - learning_rate: 0.0010
Epoch 6/200
22/22 - 0s - 15ms/step - loss: 0.0036 - val_loss: 8.7346e-04 - learning_rate: 0.0010
Epoch 7/200
22/22 - 1s - 28ms/step - loss: 0.0034 - val_loss: 6.5828e-04 - learning_rate: 0.0010
Epoch 8/200
22/22 - 1s - 28ms/step - loss: 0.0035 - val_loss: 6.5239e-04 - learning_rate: 0.0010
Epoch 9/200
22/22 - 1s - 28ms/step - loss: 0.0031 - val_loss: 5.9360e-04 - learning_rate: 2.0000e-04
Epoch 10/200
22/22 - 1s - 28ms/step - loss: 0.0030 - val_loss: 6.1415e-04 - learning_rate: 2.0000e-04
Epoch 11/200
22/22 - 0s - 15ms/step 

...

22/22 - 6s - 253ms/step - loss: 0.0658 - val_loss: 0.0119 - learning_rate: 0.0010
Epoch 2/200
22/22 - 1s - 37ms/step - loss: 0.0114 - val_loss: 0.0010 - learning_rate: 0.0010
Epoch 3/200
22/22 - 0s - 14ms/step - loss: 0.0061 - val_loss: 6.6675e-04 - learning_rate: 0.0010
Epoch 4/200
22/22 - 0s - 16ms/step - loss: 0.0059 - val_loss: 0.0012 - learning_rate: 0.0010
Epoch 5/200
22/22 - 0s - 15ms/step - loss: 0.0055 - val_loss: 7.0869e-04 - learning_rate: 0.0010
Epoch 6/200
22/22 - 0s - 15ms/step - loss: 0.0049 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 7/200
22/22 - 0s - 14ms/step - loss: 0.0050 - val_loss: 6.8828e-04 - learning_rate: 0.0010
Epoch 8/200
22/22 - 1s - 29ms/step - loss: 0.0048 - val_loss: 6.6158e-04 - learning_rate: 0.0010
Epoch 9/200
22/22 - 0s - 15ms/step - loss: 0.0042 - val_loss: 7.1467e-04 - learning_rate: 2.0000e-04
Epoch 10/200
22/22 - 1s - 27ms/step - loss: 0.0047 - val_loss: 6.7740e-04 - learning_rate: 2.0000e-04
Epoch 11/200
22/22 - 1s - 29ms/step - loss: 0.00

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


22/22 - 4s - 197ms/step - loss: 0.0135 - val_loss: 0.0259 - learning_rate: 0.0010
Epoch 2/200
22/22 - 1s - 38ms/step - loss: 0.0022 - val_loss: 0.0028 - learning_rate: 0.0010
Epoch 3/200
22/22 - 1s - 37ms/step - loss: 0.0014 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 4/200
22/22 - 1s - 27ms/step - loss: 0.0015 - val_loss: 0.0022 - learning_rate: 0.0010
Epoch 5/200
22/22 - 1s - 29ms/step - loss: 0.0012 - val_loss: 0.0017 - learning_rate: 0.0010
Epoch 6/200
22/22 - 1s - 26ms/step - loss: 0.0012 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 7/200
22/22 - 1s - 27ms/step - loss: 0.0012 - val_loss: 0.0024 - learning_rate: 0.0010
Epoch 8/200
22/22 - 0s - 22ms/step - loss: 0.0011 - val_loss: 0.0020 - learning_rate: 0.0010
Epoch 9/200
22/22 - 0s - 15ms/step - loss: 0.0010 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 10/200
22/22 - 0s - 15ms/step - loss: 0.0011 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 11/200
22/22 - 1s - 28ms/step - loss: 0.0010 - val_loss: 0.0015 - learning

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


22/22 - 4s - 196ms/step - loss: 0.0469 - val_loss: 0.0043 - learning_rate: 0.0010
Epoch 2/200
22/22 - 1s - 49ms/step - loss: 0.0088 - val_loss: 0.0062 - learning_rate: 0.0010
Epoch 3/200
22/22 - 1s - 30ms/step - loss: 0.0073 - val_loss: 0.0067 - learning_rate: 0.0010
Epoch 4/200
22/22 - 1s - 26ms/step - loss: 0.0071 - val_loss: 0.0045 - learning_rate: 0.0010
Epoch 5/200
22/22 - 1s - 25ms/step - loss: 0.0062 - val_loss: 0.0036 - learning_rate: 0.0010
Epoch 6/200
22/22 - 1s - 31ms/step - loss: 0.0062 - val_loss: 0.0016 - learning_rate: 0.0010
Epoch 7/200
22/22 - 1s - 26ms/step - loss: 0.0071 - val_loss: 0.0021 - learning_rate: 0.0010
Epoch 8/200
22/22 - 0s - 20ms/step - loss: 0.0063 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 9/200
22/22 - 0s - 14ms/step - loss: 0.0054 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 10/200
22/22 - 1s - 31ms/step - loss: 0.0057 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 11/200
22/22 - 1s - 26ms/step - loss: 0.0056 - val_loss: 0.0013 - learning

In [11]:
cnn_models = {}

for stock_symbol in stock_symbols:
    data = stock_historical_data(stock_symbol, start_date=four_years_ago.strftime('%Y-%m-%d'), end_date=today.strftime('%Y-%m-%d'))

    if data.empty:
        continue

    df_pandas = pd.DataFrame(data)[['time', 'close']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})
    df_pandas['date'] = pd.to_datetime(df_pandas['date'])
    df_pandas = df_pandas.sort_values(by='date')

    prices = df_pandas['close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)

    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]

    X_train, y_train = create_dataset(train_data)
    X_test, y_test = create_dataset(test_data)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    cnn_model = Sequential()
    cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(X_train.shape[1], 1)))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.2))

    cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.2))

    cnn_model.add(Flatten())
    cnn_model.add(Dense(50, activation='relu'))
    cnn_model.add(Dense(1))
    cnn_model.compile(loss='mean_squared_error', optimizer='adam')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    cnn_model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2, validation_data=(X_test, y_test), callbacks=[early_stopping])

    y_pred_cnn = cnn_model.predict(X_test)
    y_test_inv_cnn = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv_cnn = scaler.inverse_transform(y_pred_cnn)

    rmse_cnn = np.sqrt(mean_squared_error(y_test_inv_cnn, y_pred_inv_cnn))
    mae_cnn = mean_absolute_error(y_test_inv_cnn, y_pred_inv_cnn)
    r_squared_cnn = r2_score(y_test_inv_cnn, y_pred_inv_cnn)
    mape_cnn = np.mean(np.abs((y_test_inv_cnn - y_pred_inv_cnn) / y_test_inv_cnn)) * 100

    cnn_models[stock_symbol] = {
        'model': cnn_model,
        'y_test': y_test_inv_cnn.flatten(),
        'y_pred': y_pred_inv_cnn.flatten(),
        'dates': df_pandas['date'].values[-len(y_test):],
        'rmse': rmse_cnn,
        'mae': mae_cnn,
        'r_squared': r_squared_cnn,
        'mape': mape_cnn
    }


Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


Epoch 1/200
22/22 - 3s - 122ms/step - loss: 0.7347 - val_loss: 0.5238
Epoch 2/200
22/22 - 0s - 6ms/step - loss: 0.2746 - val_loss: 0.5293
Epoch 3/200
22/22 - 0s - 6ms/step - loss: 0.1707 - val_loss: 0.5030
Epoch 4/200
22/22 - 0s - 6ms/step - loss: 0.1338 - val_loss: 0.4528
Epoch 5/200
22/22 - 0s - 6ms/step - loss: 0.1082 - val_loss: 0.3625
Epoch 6/200
22/22 - 0s - 6ms/step - loss: 0.0856 - val_loss: 0.3482
Epoch 7/200
22/22 - 0s - 6ms/step - loss: 0.0610 - val_loss: 0.3210
Epoch 8/200
22/22 - 0s - 7ms/step - loss: 0.0540 - val_loss: 0.2825
Epoch 9/200
22/22 - 0s - 7ms/step - loss: 0.0433 - val_loss: 0.2524
Epoch 10/200
22/22 - 0s - 13ms/step - loss: 0.0383 - val_loss: 0.2543
Epoch 11/200
22/22 - 0s - 6ms/step - loss: 0.0405 - val_loss: 0.2595
Epoch 12/200
22/22 - 0s - 6ms/step - loss: 0.0339 - val_loss: 0.2315
Epoch 13/200
22/22 - 0s - 6ms/step - loss: 0.0368 - val_loss: 0.1937
Epoch 14/200
22/22 - 0s - 7ms/step - loss: 0.0375 - val_loss: 0.1734
Epoch 15/200
22/22 - 0s - 6ms/step - los

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


22/22 - 3s - 114ms/step - loss: 0.6238 - val_loss: 0.0396
Epoch 2/200
22/22 - 0s - 19ms/step - loss: 0.3104 - val_loss: 0.0346
Epoch 3/200
22/22 - 0s - 6ms/step - loss: 0.1827 - val_loss: 0.0336
Epoch 4/200
22/22 - 0s - 6ms/step - loss: 0.1359 - val_loss: 0.0329
Epoch 5/200
22/22 - 0s - 6ms/step - loss: 0.0899 - val_loss: 0.0278
Epoch 6/200
22/22 - 0s - 7ms/step - loss: 0.0772 - val_loss: 0.0265
Epoch 7/200
22/22 - 0s - 6ms/step - loss: 0.0770 - val_loss: 0.0282
Epoch 8/200
22/22 - 0s - 6ms/step - loss: 0.0654 - val_loss: 0.0272
Epoch 9/200
22/22 - 0s - 6ms/step - loss: 0.0510 - val_loss: 0.0273
Epoch 10/200
22/22 - 0s - 6ms/step - loss: 0.0433 - val_loss: 0.0314
Epoch 11/200
22/22 - 0s - 6ms/step - loss: 0.0428 - val_loss: 0.0298
Epoch 12/200
22/22 - 0s - 6ms/step - loss: 0.0408 - val_loss: 0.0336
Epoch 13/200
22/22 - 0s - 5ms/step - loss: 0.0412 - val_loss: 0.0287
Epoch 14/200
22/22 - 0s - 6ms/step - loss: 0.0325 - val_loss: 0.0321
Epoch 15/200
22/22 - 0s - 5ms/step - loss: 0.0348 - 

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


...

22/22 - 3s - 150ms/step - loss: 0.4745 - val_loss: 0.2088
Epoch 2/200
22/22 - 0s - 13ms/step - loss: 0.1576 - val_loss: 0.2218
Epoch 3/200
22/22 - 0s - 12ms/step - loss: 0.0977 - val_loss: 0.2080
Epoch 4/200
22/22 - 0s - 9ms/step - loss: 0.0835 - val_loss: 0.1916
Epoch 5/200
22/22 - 0s - 6ms/step - loss: 0.0653 - val_loss: 0.1863
Epoch 6/200
22/22 - 0s - 6ms/step - loss: 0.0586 - val_loss: 0.1701
Epoch 7/200
22/22 - 0s - 7ms/step - loss: 0.0445 - val_loss: 0.1578
Epoch 8/200
22/22 - 0s - 7ms/step - loss: 0.0345 - val_loss: 0.1454
Epoch 9/200
22/22 - 0s - 13ms/step - loss: 0.0452 - val_loss: 0.1527
Epoch 10/200
22/22 - 0s - 6ms/step - loss: 0.0313 - val_loss: 0.1397
Epoch 11/200
22/22 - 0s - 6ms/step - loss: 0.0291 - val_loss: 0.1380
Epoch 12/200
22/22 - 0s - 6ms/step - loss: 0.0308 - val_loss: 0.1335
Epoch 13/200
22/22 - 0s - 6ms/step - loss: 0.0245 - val_loss: 0.1207
Epoch 14/200
22/22 - 0s - 7ms/step - loss: 0.0263 - val_loss: 0.1214
Epoch 15/200
22/22 - 0s - 13ms/step - loss: 0.0252

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


22/22 - 2s - 112ms/step - loss: 1.0154 - val_loss: 0.4498
Epoch 2/200
22/22 - 0s - 19ms/step - loss: 0.2309 - val_loss: 0.3721
Epoch 3/200
22/22 - 0s - 6ms/step - loss: 0.1462 - val_loss: 0.3423
Epoch 4/200
22/22 - 0s - 7ms/step - loss: 0.1056 - val_loss: 0.2812
Epoch 5/200
22/22 - 0s - 6ms/step - loss: 0.0792 - val_loss: 0.2326
Epoch 6/200
22/22 - 0s - 6ms/step - loss: 0.0762 - val_loss: 0.1906
Epoch 7/200
22/22 - 0s - 7ms/step - loss: 0.0572 - val_loss: 0.1685
Epoch 8/200
22/22 - 0s - 6ms/step - loss: 0.0485 - val_loss: 0.1608
Epoch 9/200
22/22 - 0s - 6ms/step - loss: 0.0471 - val_loss: 0.1658
Epoch 10/200
22/22 - 0s - 7ms/step - loss: 0.0370 - val_loss: 0.1607
Epoch 11/200
22/22 - 0s - 7ms/step - loss: 0.0404 - val_loss: 0.1580
Epoch 12/200
22/22 - 0s - 6ms/step - loss: 0.0361 - val_loss: 0.1572
Epoch 13/200
22/22 - 0s - 6ms/step - loss: 0.0315 - val_loss: 0.1461
Epoch 14/200
22/22 - 0s - 7ms/step - loss: 0.0312 - val_loss: 0.1473
Epoch 15/200
22/22 - 0s - 7ms/step - loss: 0.0227 - 

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


22/22 - 2s - 113ms/step - loss: 0.3485 - val_loss: 0.0645
Epoch 2/200
22/22 - 0s - 20ms/step - loss: 0.1294 - val_loss: 0.0515
Epoch 3/200
22/22 - 0s - 7ms/step - loss: 0.0859 - val_loss: 0.0470
Epoch 4/200
22/22 - 0s - 18ms/step - loss: 0.0679 - val_loss: 0.0412
Epoch 5/200
22/22 - 0s - 10ms/step - loss: 0.0584 - val_loss: 0.0403
Epoch 6/200
22/22 - 0s - 16ms/step - loss: 0.0457 - val_loss: 0.0445
Epoch 7/200
22/22 - 0s - 13ms/step - loss: 0.0370 - val_loss: 0.0391
Epoch 8/200
22/22 - 0s - 9ms/step - loss: 0.0349 - val_loss: 0.0545
Epoch 9/200
22/22 - 0s - 14ms/step - loss: 0.0300 - val_loss: 0.0570
Epoch 10/200
22/22 - 0s - 15ms/step - loss: 0.0263 - val_loss: 0.0565
Epoch 11/200
22/22 - 0s - 15ms/step - loss: 0.0262 - val_loss: 0.0622
Epoch 12/200
22/22 - 0s - 9ms/step - loss: 0.0236 - val_loss: 0.0580
Epoch 13/200
22/22 - 0s - 11ms/step - loss: 0.0230 - val_loss: 0.0727
Epoch 14/200
22/22 - 0s - 7ms/step - loss: 0.0201 - val_loss: 0.0861
Epoch 15/200
22/22 - 0s - 7ms/step - loss: 0

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2

def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

dnn_models = {}

for stock_symbol in stock_symbols:
    data = stock_historical_data(stock_symbol, start_date=four_years_ago.strftime('%Y-%m-%d'), end_date=today.strftime('%Y-%m-%d'))

    if data.empty:
        continue

    df_pandas = pd.DataFrame(data)[['time', 'close']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})
    df_pandas['date'] = pd.to_datetime(df_pandas['date'])
    df_pandas = df_pandas.sort_values(by='date')

    prices = df_pandas['close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)

    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]

    X_train, y_train = create_dataset(train_data, time_step=10)
    X_test, y_test = create_dataset(test_data, time_step=10)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1])

    dnn_model = Sequential()
    dnn_model.add(Dense(128, input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
    dnn_model.add(LeakyReLU(alpha=0.1))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Dropout(0.2))

    dnn_model.add(Dense(64, kernel_regularizer=l2(0.01)))
    dnn_model.add(LeakyReLU(alpha=0.1))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Dropout(0.2))

    dnn_model.add(Dense(32, kernel_regularizer=l2(0.01)))
    dnn_model.add(LeakyReLU(alpha=0.1))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Dropout(0.2))

    dnn_model.add(Dense(16, kernel_regularizer=l2(0.01)))
    dnn_model.add(LeakyReLU(alpha=0.1))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Dropout(0.2))

    dnn_model.add(Dense(1))
    dnn_model.compile(loss='mean_squared_error', optimizer='adam')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    dnn_model.fit(X_train, y_train, epochs=300, batch_size=16, verbose=2, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])

    y_pred_dnn = dnn_model.predict(X_test)
    y_test_inv_dnn = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv_dnn = scaler.inverse_transform(y_pred_dnn)

    rmse_dnn = np.sqrt(mean_squared_error(y_test_inv_dnn, y_pred_inv_dnn))
    mae_dnn = mean_absolute_error(y_test_inv_dnn, y_pred_inv_dnn)
    r_squared_dnn = r2_score(y_test_inv_dnn, y_pred_inv_dnn)
    mape_dnn = np.mean(np.abs((y_test_inv_dnn - y_pred_inv_dnn) / y_test_inv_dnn)) * 100

    dnn_models[stock_symbol] = {
        'model': dnn_model,
        'y_test': y_test_inv_dnn.flatten(),
        'y_pred': y_pred_inv_dnn.flatten(),
        'dates': df_pandas['date'].values[-len(y_test):],
        'rmse': rmse_dnn,
        'mae': mae_dnn,
        'r_squared': r_squared_dnn,
        'mape': mape_dnn
    }


Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
Argument `alpha` is deprecated. Use `negative_slope` instead.


Epoch 1/300
44/44 - 4s - 87ms/step - loss: 2.9786 - val_loss: 2.0608 - learning_rate: 0.0010
Epoch 2/300
44/44 - 0s - 9ms/step - loss: 2.3598 - val_loss: 1.8978 - learning_rate: 0.0010
Epoch 3/300
44/44 - 0s - 5ms/step - loss: 2.1664 - val_loss: 1.8197 - learning_rate: 0.0010
Epoch 4/300
44/44 - 0s - 7ms/step - loss: 1.9909 - val_loss: 1.6932 - learning_rate: 0.0010
Epoch 5/300
44/44 - 0s - 5ms/step - loss: 1.8380 - val_loss: 1.5804 - learning_rate: 0.0010
Epoch 6/300
44/44 - 0s - 7ms/step - loss: 1.7035 - val_loss: 1.5108 - learning_rate: 0.0010
Epoch 7/300
44/44 - 0s - 6ms/step - loss: 1.6632 - val_loss: 1.4307 - learning_rate: 0.0010
Epoch 8/300
44/44 - 0s - 5ms/step - loss: 1.5772 - val_loss: 1.3536 - learning_rate: 0.0010
Epoch 9/300
44/44 - 0s - 6ms/step - loss: 1.5276 - val_loss: 1.3502 - learning_rate: 0.0010
Epoch 10/300
44/44 - 0s - 7ms/step - loss: 1.4251 - val_loss: 1.2244 - learning_rate: 0.0010
Epoch 11/300
44/44 - 0s - 6ms/step - loss: 1.3785 - val_loss: 1.2105 - learnin

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
Argument `alpha` is deprecated. Use `negative_slope` instead.


Epoch 1/300
44/44 - 4s - 88ms/step - loss: 3.3470 - val_loss: 1.6769 - learning_rate: 0.0010
Epoch 2/300
44/44 - 0s - 7ms/step - loss: 2.6115 - val_loss: 1.6451 - learning_rate: 0.0010
Epoch 3/300
44/44 - 0s - 6ms/step - loss: 2.1503 - val_loss: 1.6070 - learning_rate: 0.0010
Epoch 4/300
44/44 - 0s - 5ms/step - loss: 1.9871 - val_loss: 1.5655 - learning_rate: 0.0010
Epoch 5/300
44/44 - 0s - 7ms/step - loss: 1.9279 - val_loss: 1.5266 - learning_rate: 0.0010
Epoch 6/300
44/44 - 0s - 7ms/step - loss: 1.8557 - val_loss: 1.4833 - learning_rate: 0.0010
Epoch 7/300
44/44 - 0s - 5ms/step - loss: 1.6817 - val_loss: 1.4360 - learning_rate: 0.0010
Epoch 8/300
44/44 - 0s - 4ms/step - loss: 1.6346 - val_loss: 1.3872 - learning_rate: 0.0010
Epoch 9/300
44/44 - 0s - 5ms/step - loss: 1.5931 - val_loss: 1.3409 - learning_rate: 0.0010
Epoch 10/300
44/44 - 0s - 7ms/step - loss: 1.4936 - val_loss: 1.2919 - learning_rate: 0.0010
Epoch 11/300
44/44 - 0s - 4ms/step - loss: 1.4293 - val_loss: 1.2466 - learnin

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
Argument `alpha` is deprecated. Use `negative_slope` instead.


...

Epoch 1/300
44/44 - 4s - 87ms/step - loss: 3.1165 - val_loss: 1.6662 - learning_rate: 0.0010
Epoch 2/300
44/44 - 0s - 8ms/step - loss: 2.3546 - val_loss: 1.6015 - learning_rate: 0.0010
Epoch 3/300
44/44 - 0s - 8ms/step - loss: 2.1792 - val_loss: 1.5508 - learning_rate: 0.0010
Epoch 4/300
44/44 - 0s - 7ms/step - loss: 1.9905 - val_loss: 1.5073 - learning_rate: 0.0010
Epoch 5/300
44/44 - 0s - 5ms/step - loss: 1.8807 - val_loss: 1.4486 - learning_rate: 0.0010
Epoch 6/300
44/44 - 0s - 8ms/step - loss: 1.7227 - val_loss: 1.3915 - learning_rate: 0.0010
Epoch 7/300
44/44 - 0s - 8ms/step - loss: 1.6265 - val_loss: 1.3399 - learning_rate: 0.0010
Epoch 8/300
44/44 - 0s - 8ms/step - loss: 1.5579 - val_loss: 1.2888 - learning_rate: 0.0010
Epoch 9/300
44/44 - 1s - 14ms/step - loss: 1.4927 - val_loss: 1.2372 - learning_rate: 0.0010
Epoch 10/300
44/44 - 1s - 14ms/step - loss: 1.3958 - val_loss: 1.1869 - learning_rate: 0.0010
Epoch 11/300
44/44 - 0s - 8ms/step - loss: 1.3570 - val_loss: 1.1387 - learn

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
Argument `alpha` is deprecated. Use `negative_slope` instead.


Epoch 1/300
44/44 - 4s - 86ms/step - loss: 2.6200 - val_loss: 1.8488 - learning_rate: 0.0010
Epoch 2/300
44/44 - 0s - 6ms/step - loss: 2.1714 - val_loss: 1.7229 - learning_rate: 0.0010
Epoch 3/300
44/44 - 0s - 5ms/step - loss: 1.9497 - val_loss: 1.6781 - learning_rate: 0.0010
Epoch 4/300
44/44 - 0s - 5ms/step - loss: 1.7951 - val_loss: 1.5801 - learning_rate: 0.0010
Epoch 5/300
44/44 - 0s - 4ms/step - loss: 1.6758 - val_loss: 1.4912 - learning_rate: 0.0010
Epoch 6/300
44/44 - 0s - 4ms/step - loss: 1.5625 - val_loss: 1.4304 - learning_rate: 0.0010
Epoch 7/300
44/44 - 0s - 8ms/step - loss: 1.4858 - val_loss: 1.3768 - learning_rate: 0.0010
Epoch 8/300
44/44 - 0s - 7ms/step - loss: 1.3962 - val_loss: 1.2841 - learning_rate: 0.0010
Epoch 9/300
44/44 - 0s - 7ms/step - loss: 1.3102 - val_loss: 1.2084 - learning_rate: 0.0010
Epoch 10/300
44/44 - 1s - 15ms/step - loss: 1.2413 - val_loss: 1.1144 - learning_rate: 0.0010
Epoch 11/300
44/44 - 1s - 14ms/step - loss: 1.1818 - val_loss: 1.0654 - learn

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
Argument `alpha` is deprecated. Use `negative_slope` instead.


Epoch 1/300
44/44 - 4s - 90ms/step - loss: 3.5183 - val_loss: 1.6931 - learning_rate: 0.0010
Epoch 2/300
44/44 - 0s - 9ms/step - loss: 2.5296 - val_loss: 1.7082 - learning_rate: 0.0010
Epoch 3/300
44/44 - 0s - 6ms/step - loss: 2.3199 - val_loss: 1.6723 - learning_rate: 0.0010
Epoch 4/300
44/44 - 0s - 7ms/step - loss: 2.0921 - val_loss: 1.6369 - learning_rate: 0.0010
Epoch 5/300
44/44 - 0s - 5ms/step - loss: 1.9602 - val_loss: 1.5781 - learning_rate: 0.0010
Epoch 6/300
44/44 - 0s - 7ms/step - loss: 1.8332 - val_loss: 1.5397 - learning_rate: 0.0010
Epoch 7/300
44/44 - 0s - 6ms/step - loss: 1.7431 - val_loss: 1.4759 - learning_rate: 0.0010
Epoch 8/300
44/44 - 0s - 5ms/step - loss: 1.6605 - val_loss: 1.3956 - learning_rate: 0.0010
Epoch 9/300
44/44 - 0s - 4ms/step - loss: 1.5747 - val_loss: 1.3145 - learning_rate: 0.0010
Epoch 10/300
44/44 - 0s - 8ms/step - loss: 1.5041 - val_loss: 1.2420 - learning_rate: 0.0010
Epoch 11/300
44/44 - 0s - 8ms/step - loss: 1.4263 - val_loss: 1.2386 - learnin

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

def create_dataset(dataset, look_back=5):
    X, y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(y)

def update_and_train_model(models_dict, stock_symbol, new_data, model_type):
    new_data['date'] = pd.to_datetime(new_data['date'])
    new_data = new_data.sort_values(by='date')

    if stock_symbol in models_dict:
        model_info = models_dict[stock_symbol]
        model = model_info['model']
        scaler = model_info['scaler']
        df_pandas = model_info['df_pandas']
    else:
        print(f"Mô hình {model_type} cho mã cổ phiếu {stock_symbol} chưa được khởi tạo.")
        return

    df_pandas = pd.concat([df_pandas, new_data]).drop_duplicates(subset=['date'], keep='last')
    df_pandas = df_pandas.sort_values(by='date')
    prices = df_pandas['close'].values.reshape(-1, 1)
    scaled_data = scaler.fit_transform(prices)

    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]
    X_train, y_train = create_dataset(train_data)
    X_test, y_test = create_dataset(test_data)

    y_train_class = (y_train[1:] > y_train[:-1]).astype(int)
    y_test_class = (y_test[1:] > y_test[:-1]).astype(int)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    model.fit(X_train, y_train_class, epochs=50, batch_size=16, verbose=2, validation_data=(X_test, y_test_class), callbacks=[early_stopping, reduce_lr])

    y_pred_class = (model.predict(X_test) > 0.5).astype(int)

    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv = scaler.inverse_transform(model.predict(X_test))

    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r_squared = r2_score(y_test_inv, y_pred_inv)

    # Now only store the regression metrics
    models_dict[stock_symbol] = {
        'model': model,
        'y_test': y_test_inv.flatten(),
        'y_pred': y_pred_inv.flatten(),
        'dates': df_pandas['date'].values[-len(y_test):],
        'rmse': rmse,
        'mae': mae,
        'r_squared': r_squared,
        'df_pandas': df_pandas,
        'scaler': scaler
    }

def update_all_models(stock_symbol, new_data):
    update_and_train_model(models, stock_symbol, new_data, 'LSTM')
    update_and_train_model(gru_models, stock_symbol, new_data, 'GRU')
    update_and_train_model(cnn_models, stock_symbol, new_data, 'CNN')
    update_and_train_model(dnn_models, stock_symbol, new_data, 'DNN')


In [14]:
from google.colab import drive
import os
import pickle
import shutil

drive.mount('/content/drive')

save_dir = "/content/drive/My Drive/Model_storage"
os.makedirs(save_dir, exist_ok=True)

def save_model_data(model_dict, model_type):
    model_dir = os.path.join(save_dir, model_type)
    os.makedirs(model_dir, exist_ok=True)

    for stock_symbol, model_data in model_dict.items():
        file_path = os.path.join(model_dir, f"{model_type}_model_{stock_symbol}.pkl")
        with open(file_path, 'wb') as file:
            pickle.dump(model_data, file)

    zip_file_path = os.path.join(save_dir, f"{model_type}_models.zip")
    shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', model_dir)
    print(f"All {model_type} models have been saved and compressed to {zip_file_path}")

save_model_data(models, "LSTM")

save_model_data(gru_models, "GRU")

save_model_data(cnn_models, "CNN")

save_model_data(dnn_models, "DNN")


Mounted at /content/drive
All LSTM models have been saved and compressed to /content/drive/My Drive/Model_storage/LSTM_models.zip
All GRU models have been saved and compressed to /content/drive/My Drive/Model_storage/GRU_models.zip
All CNN models have been saved and compressed to /content/drive/My Drive/Model_storage/CNN_models.zip
All DNN models have been saved and compressed to /content/drive/My Drive/Model_storage/DNN_models.zip


In [15]:
!pip install gdown

import gdown
import zipfile
import os
import pickle

storage_dir = "/content/Model_storage"

os.makedirs(storage_dir, exist_ok=True)

def download_file(file_id, output_path):
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, output_path, quiet=False)

def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def load_model_data(model_type):
    model_dir = os.path.join(storage_dir, model_type)
    model_dict = {}

    for file_name in os.listdir(model_dir):
        if file_name.endswith(".pkl"):
            stock_symbol = file_name.split("_")[-1].replace(".pkl", "")
            file_path = os.path.join(model_dir, file_name)
            with open(file_path, 'rb') as file:
                model_dict[stock_symbol] = pickle.load(file)

    print(f"All {model_type} models have been loaded.")
    return model_dict

lstm_file_id = '10eo2UHTAsrYAYkK_ol4jGTfAm_ksv3Vx'
gru_file_id = '11nJdwjaVRiVRGljMqRr_y4hNbVCRMob1'
cnn_file_id = '11s5-2wP8gMEVYuSuaUm33q_lr0zb0cJZ'
dnn_file_id = '13luqEStbbE-6bdXiZ8POssVXDS46Lxge'

lstm_zip_path = os.path.join(storage_dir, 'LSTM_models.zip')
gru_zip_path = os.path.join(storage_dir, 'GRU_models.zip')
cnn_zip_path = os.path.join(storage_dir, 'CNN_models.zip')
dnn_zip_path = os.path.join(storage_dir, 'DNN_models.zip')

download_file(lstm_file_id, lstm_zip_path)
unzip_file(lstm_zip_path, os.path.join(storage_dir, 'LSTM'))

download_file(gru_file_id, gru_zip_path)
unzip_file(gru_zip_path, os.path.join(storage_dir, 'GRU'))

download_file(cnn_file_id, cnn_zip_path)
unzip_file(cnn_zip_path, os.path.join(storage_dir, 'CNN'))

download_file(dnn_file_id, dnn_zip_path)
unzip_file(dnn_zip_path, os.path.join(storage_dir, 'DNN'))

lstm_models = load_model_data("LSTM")
gru_models = load_model_data("GRU")
cnn_models = load_model_data("CNN")
dnn_models = load_model_data("DNN")

Downloading...
From (original): https://drive.google.com/uc?id=10eo2UHTAsrYAYkK_ol4jGTfAm_ksv3Vx
From (redirected): https://drive.google.com/uc?id=10eo2UHTAsrYAYkK_ol4jGTfAm_ksv3Vx&confirm=t&uuid=1e10fe1d-dcb8-4e5b-b9db-c563e6a74813
To: /content/Model_storage/LSTM_models.zip
100%|██████████| 41.3M/41.3M [00:00<00:00, 147MB/s]
Downloading...
From: https://drive.google.com/uc?id=11nJdwjaVRiVRGljMqRr_y4hNbVCRMob1
To: /content/Model_storage/GRU_models.zip
100%|██████████| 19.9M/19.9M [00:00<00:00, 178MB/s]
Downloading...
From: https://drive.google.com/uc?id=11s5-2wP8gMEVYuSuaUm33q_lr0zb0cJZ
To: /content/Model_storage/CNN_models.zip
100%|██████████| 2.07M/2.07M [00:00<00:00, 119MB/s]
Downloading...
From: https://drive.google.com/uc?id=13luqEStbbE-6bdXiZ8POssVXDS46Lxge
To: /content/Model_storage/DNN_models.zip
100%|██████████| 3.15M/3.15M [00:00<00:00, 137MB/s]


All LSTM models have been loaded.
All GRU models have been loaded.
All CNN models have been loaded.
All DNN models have been loaded.


In [16]:
import plotly.graph_objects as go
from IPython.display import clear_output
import ipywidgets as widgets
import datetime

def display_technical_chart(stock_symbol):
    clear_output(wait=True)
    print(f"Hiển thị biểu đồ kỹ thuật cho cổ phiếu: {stock_symbol}")
    plot_technical_indicators(stock_symbol)
    display(reset_button)

def display_prediction_chart(stock_symbol):
    clear_output(wait=True)
    print(f"Hiển thị dữ liệu dự đoán cho cổ phiếu: {stock_symbol}")

    fig = go.Figure()
    metrics_summary = {}

    if stock_symbol in lstm_models:
        y_test_lstm = lstm_models[stock_symbol]['y_test']
        y_pred_lstm = lstm_models[stock_symbol]['y_pred']
        dates = lstm_models[stock_symbol]['dates']

        fig.add_trace(go.Scatter(x=dates, y=y_test_lstm, mode='lines', name='Giá thực tế', line=dict(color='blue', width=2, shape='spline')))
        fig.add_trace(go.Scatter(x=dates, y=y_pred_lstm, mode='lines', name='Giá dự đoán LSTM', line=dict(color='orange', shape='spline')))

        metrics_summary['LSTM'] = {
            'rmse': lstm_models[stock_symbol]['rmse'],
            'mae': lstm_models[stock_symbol]['mae'],
            'r_squared': lstm_models[stock_symbol]['r_squared'],
            'mape': lstm_models[stock_symbol]['mape']
        }

    if stock_symbol in gru_models:
        y_test_gru = gru_models[stock_symbol]['y_test']
        y_pred_gru = gru_models[stock_symbol]['y_pred']
        dates_gru = gru_models[stock_symbol]['dates']

        fig.add_trace(go.Scatter(x=dates_gru, y=y_pred_gru, mode='lines', name='Giá dự đoán GRU', line=dict(color='red', dash='dash', shape='spline')))

        metrics_summary['GRU'] = {
            'rmse': gru_models[stock_symbol]['rmse'],
            'mae': gru_models[stock_symbol]['mae'],
            'r_squared': gru_models[stock_symbol]['r_squared'],
            'mape': gru_models[stock_symbol]['mape']
        }

    if stock_symbol in cnn_models:
        y_test_cnn = cnn_models[stock_symbol]['y_test']
        y_pred_cnn = cnn_models[stock_symbol]['y_pred']
        dates_cnn = cnn_models[stock_symbol]['dates']

        fig.add_trace(go.Scatter(x=dates_cnn, y=y_pred_cnn, mode='lines', name='Giá dự đoán CNN', line=dict(color='green', dash='dot', shape='spline')))

        metrics_summary['CNN'] = {
            'rmse': cnn_models[stock_symbol]['rmse'],
            'mae': cnn_models[stock_symbol]['mae'],
            'r_squared': cnn_models[stock_symbol]['r_squared'],
            'mape': cnn_models[stock_symbol]['mape']
        }

    if stock_symbol in dnn_models:
        y_test_dnn = dnn_models[stock_symbol]['y_test']
        y_pred_dnn = dnn_models[stock_symbol]['y_pred']
        dates_dnn = dnn_models[stock_symbol]['dates']

        fig.add_trace(go.Scatter(x=dates_dnn, y=y_pred_dnn, mode='lines', name='Giá dự đoán DNN', line=dict(color='purple', dash='dashdot', shape='spline')))

        metrics_summary['DNN'] = {
            'rmse': dnn_models[stock_symbol]['rmse'],
            'mae': dnn_models[stock_symbol]['mae'],
            'r_squared': dnn_models[stock_symbol]['r_squared'],
            'mape': dnn_models[stock_symbol]['mape']
        }

    fig.update_layout(title=f'Dự đoán giá cổ phiếu {stock_symbol}',
                      xaxis_title='Ngày',
                      yaxis_title='Giá',
                      template='plotly_white')
    fig.show()

    for model_name, metrics in metrics_summary.items():
        print(f"Dự đoán và Giá thực tế ({model_name}):")
        print(f"RMSE ({model_name}): {metrics['rmse']:.2f}")
        print(f"MAE ({model_name}): {metrics['mae']:.2f}")
        print(f"R² ({model_name}): {metrics['r_squared']:.2f}")
        print(f"MAPE ({model_name}): {metrics['mape']:.2f}%")
        print()

    if metrics_summary:
        best_model = min(metrics_summary.items(), key=lambda x: x[1]['rmse'])[0]
        print(f"\033[1mMô hình hiệu quả nhất là: {best_model} với RMSE: {metrics_summary[best_model]['rmse']:.2f}\033[0m")

    display(reset_button)

stock_dropdown = widgets.Dropdown(
    options=stock_symbols,
    description='Mã cổ phiếu:',
    value=stock_symbols[0]
)

tech_button = widgets.Button(description='Hiển thị biểu đồ kỹ thuật', layout=widgets.Layout(width='200px', height='40px'))
pred_button = widgets.Button(description='Hiển thị biểu đồ dự đoán', layout=widgets.Layout(width='200px', height='40px'))
reset_button = widgets.Button(description='Đặt lại', layout=widgets.Layout(width='200px', height='40px'))

def on_tech_button_clicked(b):
    display_technical_chart(stock_dropdown.value)

def on_pred_button_clicked(b):
    display_prediction_chart(stock_dropdown.value)

def on_reset_clicked(b):
    clear_output(wait=True)
    display(stock_dropdown, tech_button, pred_button, reset_button)

display(stock_dropdown, tech_button, pred_button, reset_button)

tech_button.on_click(on_tech_button_clicked)
pred_button.on_click(on_pred_button_clicked)
reset_button.on_click(on_reset_clicked)


Dropdown(description='Mã cổ phiếu:', index=3, options=('VCB', 'VNM', 'MWG', 'VIC', 'SSI', 'DGC', 'CTD', 'FPT',…

Button(description='Hiển thị biểu đồ kỹ thuật', layout=Layout(height='40px', width='200px'), style=ButtonStyle…

Button(description='Hiển thị biểu đồ dự đoán', layout=Layout(height='40px', width='200px'), style=ButtonStyle(…

Button(description='Đặt lại', layout=Layout(height='40px', width='200px'), style=ButtonStyle())